In [1]:
import logging
import pandas as pd
from winterapi import WinterAPI
from wintertoo.models import WinterFieldToO, WinterRaDecToO

In [2]:
logging.getLogger("winterapi").setLevel("DEBUG")

In [3]:
winter = WinterAPI()

# Building a ToO

Using our knowledge from the previous notebook, let's build a ToO request covering two adjacent fields:

In [4]:
too_1 = WinterFieldToO(field_id=55)
too_2 = WinterFieldToO(field_id=56)

too_list = [too_1, too_2]
too_list

[WinterFieldToO(field_id=55, filters=['Y', 'J', 'Hs'], target_priority=50.0, t_exp=30.0, n_exp=1, n_dither=1, dither_distance=30.0, start_time_mjd=60341.08759327358, end_time_mjd=60342.077593278416, max_airmass=2.0),
 WinterFieldToO(field_id=56, filters=['Y', 'J', 'Hs'], target_priority=50.0, t_exp=30.0, n_exp=1, n_dither=1, dither_distance=30.0, start_time_mjd=60341.08759327358, end_time_mjd=60342.077593278416, max_airmass=2.0)]

# Building a schedule

**If you want to build a ToO schedule, you need both a set of observations AND infornation about the program to trigger with!**

We can use one of the programs we triggered earlier:

In [5]:
print(f"Available programs: {winter.get_programs()}")

Available programs: ['202XA000']


When we add a new program locally, the API is used to look up the details of that program. We store that locally:

In [ ]:
winter.get_program_details("202XA000")

Using this information, we can 'simulate' what the API will do when building a schedule:

In [7]:
local_schedule = winter.build_schedule_locally(
    program_name="202XA000",
    data=too_list
)
local_schedule

,raDeg,decDeg,fieldID,filter,visitExpTime,priority,progPI,progName,progID,validStart,validStop,observed,maxAirmass,ditherNumber,ditherStepSize,obsHistID
0,259.2,86.0,55,Y,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,0
1,259.2,86.0,55,J,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,1
2,259.2,86.0,55,Hs,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,2
3,273.6,86.0,56,Y,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,3
4,273.6,86.0,56,J,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,4
5,273.6,86.0,56,Hs,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,5


This process involves a lot of additional validation, for example ensuring targets are visible in the time range, that the program is active, and many other things.

**If you are submitting a ToO, you should always take a look at a local copy first!**

Now let's try submitting a ToO request via the API. 

**Note, by default, the API just checks a ToO but does not actually trigger on it**

In [8]:
api_res, api_schedule = winter.submit_too(
    program_name="202XA000",
    data=too_list,
    submit_trigger=False
)

In [9]:
print(api_res)
print(api_res.json()["msg"])
api_schedule

<Response [200]>
Request passed validation! Triggering was set to False


,raDeg,decDeg,fieldID,filter,visitExpTime,priority,progPI,progName,progID,validStart,validStop,observed,maxAirmass,ditherNumber,ditherStepSize,obsHistID
0,259.2,86.0,55,Y,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,0
1,259.2,86.0,55,J,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,1
2,259.2,86.0,55,Hs,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,2
3,273.6,86.0,56,Y,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,3
4,273.6,86.0,56,J,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,4
5,273.6,86.0,56,Hs,30.0,50.0,Stein,202XA000,1,60341.087593,60342.077593,False,2.0,1,30.0,5


This looks a lot like the schedule from earlier!

As expected, when we look at the API message, we see that the schedule did not actually trigger.

**Always check that your schedule is correct, then set triggering to True, and confirm that the API returns a successful submission message!**

In [10]:
# An example ToO that would submit for real!
#
# api_res, api_schedule = winter.submit_too(
#     program_name="202XA000",
#     data=too_list,
#     submit_trigger=True
# )
# print(api_res, api_res.json()["msg"])